## Word2Vec

### trained gensim.models.Word2Vec with df['Desc'], created a visualization,reduced the dimantion with PCA to 15 and created a new dataframe with the new 15 features and the original dataframe.


#### Train

In [1]:
import gensim.models
corpus = df['Desc']
model = gensim.models.Word2Vec(corpus)

C:\Users\Hana Baron\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


NameError: name 'df' is not defined

In [ ]:
vec_king = model.wv['software']
vec_king

In [ ]:
model.wv.index_to_key

In [ ]:
for index, word in enumerate(model.wv.index_to_key):
    if index == 10:
        break
    print("index is " , index, "word is ", word)

In [ ]:
# Filter the list of vectors to include only those that Word2Vec has a vector for
vector_list = [model.wv[word] for word in words if word in model.wv.index_to_key]

# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.wv.index_to_key]

# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)

# Cast to a dict so we can turn it into a DataFrame
word_vec_dict = dict(word_vec_zip)
df_vec = pd.DataFrame.from_dict(word_vec_dict, orient='index')

#### Visualization

In [ ]:
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing

In [ ]:
######visualization of word2vec
# %matplotlib inline  
# %matplotlib ipympl
# %matplotlib notebook
# %pylab

word = "software"
fig = plt.figure(figsize = (15,15))
## word embedding
tot_words = [word] + [tupla[0] for tupla in model.wv.most_similar(word, topn=20)]
X = model.wv[tot_words]
## pca to reduce dimensionality from 300 to 3
pca = manifold.TSNE(perplexity=40, n_components=3, init='pca')
X = pca.fit_transform(X)
## create dtf
dtf_ = pd.DataFrame(X, index=tot_words, columns=["x","y","z"])
dtf_["input"] = 0
dtf_["input"].iloc[0:1] = 1

## plot 3d
from mpl_toolkits.mplot3d import Axes3D
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dtf_[dtf_["input"]==0]['x'], 
           dtf_[dtf_["input"]==0]['y'], 
           dtf_[dtf_["input"]==0]['z'], c="black")
ax.scatter(dtf_[dtf_["input"]==1]['x'], 
           dtf_[dtf_["input"]==1]['y'], 
           dtf_[dtf_["input"]==1]['z'], c="red")
ax.set(xlabel=None, ylabel=None, zlabel=None, xticklabels=[], 
       yticklabels=[], zticklabels=[])
for label, row in dtf_[["x","y","z"]].iterrows():
    x, y, z = row
    ax.text(x, y, z, s=label)
# ax.mouse_init()

#### Reducing dimantions with PCA and creating new dataframe

In [ ]:
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

# Function that will help us drop documents that have no word vectors in word2vec
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.wv.index_to_key for word in doc)

# Filter out documents
def filter_docs(corpus, texts, condition_on_doc):
    """
    Filter corpus and texts given the function condition_on_doc which takes a doc. The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [ ]:
# Remove docs that don't include any words in W2V's vocab
corpus, desc_list = filter_docs(corpus, df['Desc'], lambda doc: has_vector_representation(model, doc))

# Filter out any empty docs
corpus, desc_list = filter_docs(corpus, desc_list, lambda doc: (len(doc) != 0))
x = []
for doc in corpus: # append the vector for each document
    x.append(document_vector(model, doc))
    
X = np.array(x) # list to array

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=15, random_state=10)

# as a reminder, x is the array with our 300-dimensional vectors
reduced_vecs = pca.fit_transform(x)
df_w_vectors = pd.DataFrame(reduced_vecs)

# Compose dataframe 
df_w_vectors['Desc'] = df['Desc']
# Use pd.concat to match original titles with their vectors
main_w_vectors = pd.concat((df_w_vectors, df), axis=1)

# Get rid of vectors that couldn't be matched with the main_df
main_w_vectors.dropna(axis=0, inplace=True)

In [ ]:
main_w_vectors